In [1]:
import os
import json
import bson
import kafka_consumer
import kafka_producer
from typing import Dict

import grpc
import ner_pb2
import ner_pb2_grpc
import sa_pb2
import sa_pb2_grpc

In [15]:


def publish_result(chunk_event: Dict, ner_results: Dict, sa_results: Dict) -> None:
    chunk_value = json.loads(chunk_event.value)
    producer.send(
        os.environ.get('TOPIC_CHUNKS'),
        key=chunk_event.key,
        value=json.dumps({
            "chunk_id": chunk_value["chunk_id"],
            "project_id": chunk_value["project_id"],
            "text": chunk_value["text"],
            "text_metadata": chunk_value["text_metadata"],
            "ner_results": json.dumps(ner_results),
            "sa_results": json.dumps(sa_results)
        })
    )


# ner grpc client
ner_channel = grpc.insecure_channel(
    os.environ.get("NER_SERVER_ADDRESS")
)
ner_stub = ner_pb2_grpc.FinanceNERStub(ner_channel)
# sa grpc client
sa_channel = grpc.insecure_channel(
    os.environ.get("SA_SERVER_ADDRESS")
)
sa_stub = sa_pb2_grpc.FinanceSAStub(sa_channel)

# define consumer
consumer = kafka_consumer.initialize(
    topic_env='TOPIC_CHUNKS',
    group_id_env='CONSUMER_GROUP_CHUNKS'
)

# define kafka producer
producer = kafka_producer.initialize()



In [19]:
bson.objectid.ObjectId()

ObjectId('61d5aefab802ed001aff6e25')

In [17]:
for event in consumer:
    chunk = json.loads(event.value)
    ner_response = ner_stub.ExtractEntities(
        ner_pb2.NerRequest(
            text=chunk["text"],
            metadata=chunk["text_metadata"]
        )
    )
    ner_results = json.loads(ner_response.results)
    if ner_results:
        sa_response = sa_stub.SentimentAnalysis(
            sa_pb2.SaRequest(
                text=chunk["text"],
                metadata=chunk["text_metadata"]
            )
        )
        sa_results = json.loads(sa_response.results)
        print(event.value)
        publish_result(
            chunk_event=event,
            ner_results=ner_results,
            sa_results=sa_results
        )

{"chunk_id": "61d57ee0b1313000013358f0", "project_id": "000000000000000000000000", "text": "Don't get me wrong. I want this to work. \n\nEv cars are out of my price range for what I would get, but at least they, at the moment, are JUST out of my price range. \n\nIf they came out with an EV vehicle that can handle off road obsticles, like rivers and that level of moisture, I would change much about my life to be able to afford one.   I would even over look the issue of vehicle weight to get one. \n\nSo far, I haven't seen an EV that can handle anything that an ICE vehicle with a snorkel can handle. \n\nEV fires are also a thing.  \n\nI am 100% behind EV.  I'd love to seen it moved into the farming sector, but where are the EV tractors, and the EV plows?\n\nEdit: if yeah.  Could you provide an example of how an EV is better at a fresh or salt water crossing than and ICE vehicle?  I respect your opinion, and I think one day we will see one that can, but I have not seen an example of this 

{"chunk_id": "61d57eddb1313000013357fe", "project_id": "000000000000000000000000", "text": "BAC wrote that about the S&P 500, not large tech only. Here's [their data](https://i.redd.it/7sik5xmmq6p61.jpg), going back to 1987. People may not agree with it, but it's not like their conclusion was made up out of thin air. They published this outlook almost a year ago.", "text_metadata": "{\"timestamp\": 1641374408.0, \"comment_id\": \"hrc0yye\", \"post_id\": \"rwenx8\"}", "ner_results": "{\"BAC\": {\"span\": [1, 3]}}", "sa_results": "{\"label\": \"neutral\", \"score\": 0.9998664855957031}"}
{"chunk_id": "61d57eddb131300001335805", "project_id": "000000000000000000000000", "text": "I just can't see MSFT returning 2% a year for 10 years.  Laughable.", "text_metadata": "{\"timestamp\": 1641367087.0, \"comment_id\": \"hrbr1x7\", \"post_id\": \"rwenx8\"}", "ner_results": "{\"MSFT\": {\"span\": [17, 21]}}", "sa_results": "{\"label\": \"positive\", \"score\": 0.9792259931564331}"}
{"chunk_id": "61

{"chunk_id": "61d57eebb131300001335964", "project_id": "000000000000000000000000", "text": "There are headwinds for sure, but this is an incredible business and will continue to do phenomenal. I have about a 3% position, this is definitely a long term play for me, not a 2022. Like a 2032 or 2042. They have incredible IP and nobody does entertainment like Disney. You\u2019re totally right on parks and cruises, probably won\u2019t be recovered till mid 22 or even 2023", "text_metadata": "{\"timestamp\": 1641351977.0, \"comment_id\": \"hraymcr\", \"post_id\": \"rvxtwt\"}", "ner_results": "{\"IP\": {\"span\": [227, 229]}}", "sa_results": "{\"label\": \"positive\", \"score\": 0.9979983568191528}"}
{"chunk_id": "61d57eebb131300001335965", "project_id": "000000000000000000000000", "text": "How is it a bust? Since the beginning they set expectations that it wouldn't be profitable until around 2024, I believe. Right now it can really only be judged on subscriber numbers. I'm an AAPL investor, b

{"chunk_id": "61d57eddb13130000133582f", "project_id": "000000000000000000000000", "text": "I\u2019d pick up some AAPL and AMZN.  Both strong assets that will prob be worth more in 4 to 5 years than it is today.", "text_metadata": "{\"timestamp\": 1641357545.0, \"comment_id\": \"hrbany3\", \"post_id\": \"rwe4m3\"}", "ner_results": "{\"AMZN\": {\"span\": [26, 30]}, \"AAPL\": {\"span\": [17, 21]}}", "sa_results": "{\"label\": \"positive\", \"score\": 0.9994128942489624}"}
{"chunk_id": "61d57ee0b1313000013358b3", "project_id": "000000000000000000000000", "text": "But they already sell a shit ton of trucks. They dominate the truck market. GM and RAM will both have electric trucks within a couple years so it's not like there's no competition for this space. I don't get how converting your customers from ICE trucks to EV trucks makes them more money. Add to that F has been losing market share since 2011. \n\nI think F will be bag holder city - just IMO.", "text_metadata": "{\"timestamp\": 16

{"chunk_id": "61d57eddb1313000013357fe", "project_id": "000000000000000000000000", "text": "BAC wrote that about the S&P 500, not large tech only. Here's [their data](https://i.redd.it/7sik5xmmq6p61.jpg), going back to 1987. People may not agree with it, but it's not like their conclusion was made up out of thin air. They published this outlook almost a year ago.", "text_metadata": "{\"timestamp\": 1641374408.0, \"comment_id\": \"hrc0yye\", \"post_id\": \"rwenx8\"}", "ner_results": "{\"BAC\": {\"span\": [1, 3]}}", "sa_results": "{\"label\": \"neutral\", \"score\": 0.9998664855957031}"}
{"chunk_id": "61d57eddb131300001335805", "project_id": "000000000000000000000000", "text": "I just can't see MSFT returning 2% a year for 10 years.  Laughable.", "text_metadata": "{\"timestamp\": 1641367087.0, \"comment_id\": \"hrbr1x7\", \"post_id\": \"rwenx8\"}", "ner_results": "{\"MSFT\": {\"span\": [17, 21]}}", "sa_results": "{\"label\": \"positive\", \"score\": 0.9792259931564331}"}
{"chunk_id": "61

...

{"chunk_id": "61d57eddb131300001335825", "project_id": "000000000000000000000000", "text": "You can\u2019t go wrong with MSFT or NVDA imo. Not sure about UNH. NVDA with GPU, Self driving tech & bunch of other segments looks promising. MSFT should be the next 3T company. I personally like DIS for leaps, thinking Covid recovery at some point and + getting more into originals & expanding worldwide", "text_metadata": "{\"timestamp\": 1641359121.0, \"comment_id\": \"hrbdrkv\", \"post_id\": \"rwenjl\"}", "ner_results": "{\"MSFT\": {\"span\": [24, 28]}, \"NVDA\": {\"span\": [32, 36]}, \"DIS\": {\"span\": [192, 195]}, \"UNH\": {\"span\": [57, 60]}}", "sa_results": "{\"label\": \"positive\", \"score\": 0.998633861541748}"}
{"chunk_id": "61d57eddb131300001335826", "project_id": "000000000000000000000000", "text": "I don\u2019t care for DIS, not bearish but I\u2019d rather invest in these three. MSFT is safer imo and yes I agree next 3T. Might go for NVDA which is higher risk so should print much

{"chunk_id": "61d57edcb1313000013357be", "project_id": "000000000000000000000000", "text": "WM is not undervalued btw. Every redditor in the universe has been hyping it up the last 2 months. This means every good thing the company will do in the future is probably already priced in. \n\nIt might be a good short term trade like EV stocks or high growth tech stocks but it's most certainly not a good 10 year+ hold at the current price", "text_metadata": "{\"timestamp\": 1641374916.0, \"comment_id\": \"hrc1mkj\", \"post_id\": \"rwi3m2\"}", "ner_results": "{\"WM\": {\"span\": [1, 2]}}", "sa_results": "{\"label\": \"positive\", \"score\": 0.9913179278373718}"}
{"chunk_id": "61d57eddb1313000013357f4", "project_id": "000000000000000000000000", "text": "These are the idiots that write the headlines \u201clike buying the next TSLA AAPL at IPO\u201d", "text_metadata": "{\"timestamp\": 1641359309.0, \"comment_id\": \"hrbe4m6\", \"post_id\": \"rwenx8\"}", "ner_results": "{\"AAPL\": {\"span\": [73, 

{"chunk_id": "61d57ee0b1313000013358fe", "project_id": "000000000000000000000000", "text": "I will be messaging you in 1 day on [**2022-01-06 05:12:53 UTC**](http://www.wolframalpha.com/input/?i=2022-01-06%2005:12:53%20UTC%20To%20Local%20Time) to remind you of [**this link**](https://www.reddit.com/r/StockMarket/comments/rw1mun/is_this_the_catalyst_for_ford_what_are_ya/hrbemec/?context=3)\n\n[**CLICK THIS LINK**](https://www.reddit.com/message/compose/?to=RemindMeBot&subject=Reminder&message=%5Bhttps%3A%2F%2Fwww.reddit.com%2Fr%2FStockMarket%2Fcomments%2Frw1mun%2Fis_this_the_catalyst_for_ford_what_are_ya%2Fhrbemec%2F%5D%0A%0ARemindMe%21%202022-01-06%2005%3A12%3A53%20UTC) to send a PM to also be reminded and to reduce spam.\n\n^(Parent commenter can ) [^(delete this message to hide from others.)](https://www.reddit.com/message/compose/?to=RemindMeBot&subject=Delete%20Comment&message=Delete%21%20rw1mun)\n\n*****\n\n|[^(Info)](https://www.reddit.com/r/RemindMeBot/comments/e1bko7/remindmebo

{"chunk_id": "61d57eddb131300001335825", "project_id": "000000000000000000000000", "text": "You can\u2019t go wrong with MSFT or NVDA imo. Not sure about UNH. NVDA with GPU, Self driving tech & bunch of other segments looks promising. MSFT should be the next 3T company. I personally like DIS for leaps, thinking Covid recovery at some point and + getting more into originals & expanding worldwide", "text_metadata": "{\"timestamp\": 1641359121.0, \"comment_id\": \"hrbdrkv\", \"post_id\": \"rwenjl\"}", "ner_results": "{\"MSFT\": {\"span\": [24, 28]}, \"NVDA\": {\"span\": [32, 36]}, \"DIS\": {\"span\": [192, 195]}, \"UNH\": {\"span\": [57, 60]}}", "sa_results": "{\"label\": \"positive\", \"score\": 0.998633861541748}"}


KeyboardInterrupt: 